In [2]:
import pandas as pd

# Rutas absolutas (copiadas directamente de tu estructura)
ruta_sales = r"D:\SoyHenry\Proyecto integrador\data\sales.csv"
ruta_products = r"D:\SoyHenry\Proyecto integrador\data\products.csv"

# Cargar los datasets
sales = pd.read_csv(ruta_sales)
products = pd.read_csv(ruta_products)

# Vista previa
display(sales.head())
display(products.head())

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3


,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,1,Flour - Whole Wheat,74.2988,3,Medium,21:49.2,Durable,Unknown,0
1,2,Cookie Chocolate Chip With,91.2329,3,Medium,39:11.0,Unknown,Unknown,0
2,3,Onions - Cippolini,9.1379,9,Medium,11:51.6,Weak,FALSE,111
3,4,Sauce - Gravy; Au Jus; Mix,54.3055,9,Medium,46:28.9,Durable,Unknown,0
4,5,Artichokes - Jerusalem,65.4771,2,Low,13:35.4,Durable,TRUE,27


### Verificación de la integridad de los df para el trabajo que vamos a realizar...

In [3]:
# Verificamos nulos en columnas relevantes de ventas
sales_nulls = sales[['ProductID', 'Quantity', 'Discount']].isnull().sum()

# Nulos en precios
products_nulls = products[['ProductID', 'Price']].isnull().sum()

# Mostrar resumen
print("❗ Nulos en tabla 'sales':\n", sales_nulls)
print("\n❗ Nulos en tabla 'products':\n", products_nulls)

❗ Nulos en tabla 'sales':
 ProductID    0
Quantity     0
Discount     0
dtype: int64

❗ Nulos en tabla 'products':
 ProductID    0
Price        0
dtype: int64


In [4]:
precios_invalidos = products[products['Price'] <= 0]

print(f"❗ Productos con precio inválido (cero o negativo): {len(precios_invalidos)}")
display(precios_invalidos[['ProductID', 'ProductName', 'Price']])

❗ Productos con precio inválido (cero o negativo): 2


,ProductID,ProductName,Price
18,19,Tea - Earl Grey,0.0
154,155,Peas - Pigeon; Dry,0.0


In [5]:
descuentos_invalidos = sales[~sales['Discount'].between(0, 1)]

print(f"❗ Registros con descuentos inválidos: {len(descuentos_invalidos)}")
display(descuentos_invalidos[['SalesID', 'Discount']])

❗ Registros con descuentos inválidos: 0


,SalesID,Discount


In [6]:
# ¿Algún Quantity negativo o nulo?
cantidad_problema = sales[(sales['Quantity'] <= 0) | (sales['Quantity'].isnull())]

print(f"❗ Registros con Quantity nulo o negativo: {len(cantidad_problema)}")

❗ Registros con Quantity nulo o negativo: 0


In [7]:
# Primero identificamos los ProductID con precio 0
productos_con_precio_cero = products[products['Price'] == 0]['ProductID']

# Filtramos las ventas que corresponden a esos productos
ventas_afectadas = sales[sales['ProductID'].isin(productos_con_precio_cero)]

# Mostramos la cantidad total de registros afectados
print(f"❗ Cantidad de ventas afectadas por productos con precio cero: {len(ventas_afectadas)}")

# (Opcional) Visualizamos las primeras filas
display(ventas_afectadas.head())

❗ Cantidad de ventas afectadas por productos con precio cero: 29745


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
775,776,5,51918,19,14,0.0,0.0,2018-04-29 12:06:26.500,2YJT1LSIP7I5RUBOXP22
796,797,15,98717,19,25,0.0,0.0,2018-03-15 01:43:45.690,SEDT81CT0N28FWJLMOJB
915,916,11,87233,19,23,0.0,0.0,2018-04-14 13:35:52.700,XEQPK3NLSCAAEO5R7C9N
1164,1165,9,16843,155,5,0.0,0.0,2018-02-07 18:48:42.060,XDBR5FY4VYJ8SCCR8851
1281,1282,19,90043,19,23,0.0,0.0,2018-04-15 22:38:10.750,ILT60U3TIWM0OQWVCCOB


#### Se detectaron 29.745 registros de ventas asociadas a productos con precio cero. Estos casos fueron identificados y tratados por separado, dado que su inclusión en cálculos como ingreso total, ticket promedio o margen, distorsionaba los resultados. Se optó por excluirlos del análisis económico y documentar el impacto para asegurar la transparencia y calidad del análisis.

In [8]:
# 1. Identificamos los productos con precio cero
productos_invalidos = products[products['Price'] == 0]['ProductID']

# 2. Creamos un nuevo dataframe limpio
sales_limpias = sales[~sales['ProductID'].isin(productos_invalidos)].copy()

# 3. Confirmamos cuántos registros quedan
print(f"✅ Registros en el dataset limpio: {len(sales_limpias)}")

✅ Registros en el dataset limpio: 6728380


In [10]:
# Ruta absoluta de guardado
ruta_salida = r"D:\SoyHenry\Proyecto integrador\data\sales_limpias.csv"

# Exportar el archivo limpio
sales_limpias.to_csv(ruta_salida, index=False)

print(f"✅ Archivo guardado con éxito en:\n{ruta_salida}")

✅ Archivo guardado con éxito en:
D:\SoyHenry\Proyecto integrador\data\sales_limpias.csv


### PI 1 -------------------------------------------

In [11]:
# Unimos el dataset limpio con los precios de los productos
ventas_enriquecidas = sales_limpias.merge(
    products[['ProductID', 'Price']], on='ProductID', how='left'
)

# Aplicamos la fórmula real del precio total
ventas_enriquecidas['TotalPriceCalculated'] = (
    ventas_enriquecidas['Quantity'] * ventas_enriquecidas['Price'] * (1 - ventas_enriquecidas['Discount'])
).round(2)

# Vista previa
display(ventas_enriquecidas[['SalesID', 'ProductID', 'Quantity', 'Price', 'Discount', 'TotalPriceCalculated']].head())

,SalesID,ProductID,Quantity,Price,Discount,TotalPriceCalculated
0,1,381,7,44.2337,0.0,309.64
1,2,61,7,62.5460,0.0,437.82
2,3,23,24,79.0184,0.0,1896.44
3,4,176,19,81.3167,0.2,1236.01
4,5,310,9,79.9780,0.0,719.80


### PI 2 -------------------------------------------

In [12]:
# Cálculo de cuartiles
q1 = ventas_enriquecidas['TotalPriceCalculated'].quantile(0.25)
q3 = ventas_enriquecidas['TotalPriceCalculated'].quantile(0.75)
iqr = q3 - q1

# Límites inferior y superior para definir outliers
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

print(f"IQR: {iqr:.2f}")
print(f"Límite inferior: {limite_inferior:.2f}")
print(f"Límite superior: {limite_superior:.2f}")

IQR: 803.86
Límite inferior: -1025.20
Límite superior: 2190.24


In [13]:
ventas_enriquecidas['IsOutlier'] = ventas_enriquecidas['TotalPriceCalculated'].apply(
    lambda x: 1 if (x < limite_inferior or x > limite_superior) else 0
)

In [14]:
cantidad_outliers = ventas_enriquecidas['IsOutlier'].sum()
print(f"❗ Total de outliers detectados: {cantidad_outliers}")

❗ Total de outliers detectados: 47752


### PI 3 -------------------------------------------

In [15]:
# Aseguramos que SalesDate sea tipo datetime
ventas_enriquecidas['SalesDate'] = pd.to_datetime(ventas_enriquecidas['SalesDate'])

# Creamos la columna con la hora (de 00 a 23)
ventas_enriquecidas['HoraVenta'] = ventas_enriquecidas['SalesDate'].dt.hour

# Vista previa
ventas_enriquecidas[['SalesID', 'SalesDate', 'HoraVenta']].head()

,SalesID,SalesDate,HoraVenta
0,1,2018-02-05 07:38:25.430,7.0
1,2,2018-02-02 16:03:31.150,16.0
2,3,2018-05-03 19:31:56.880,19.0
3,4,2018-04-07 14:43:55.420,14.0
4,5,2018-02-12 15:37:03.940,15.0


In [16]:
ventas_por_hora = (
    ventas_enriquecidas.groupby('HoraVenta')['TotalPriceCalculated']
    .sum()
    .reset_index()
    .sort_values(by='TotalPriceCalculated', ascending=False)
)

display(ventas_por_hora)

hora_top = ventas_por_hora.iloc[0]
print(f"🕒 La mayor concentración de ventas ocurre a las {hora_top['HoraVenta']} hs con un total de ${hora_top['TotalPriceCalculated']:.2f}")

,HoraVenta,TotalPriceCalculated
16,16.0,1.790144e+08
20,20.0,1.789492e+08
2,2.0,1.784209e+08
6,6.0,1.783812e+08
19,19.0,1.783461e+08
0,0.0,1.783134e+08
17,17.0,1.782904e+08
9,9.0,1.781666e+08
11,11.0,1.781430e+08
15,15.0,1.780218e+08


🕒 La mayor concentración de ventas ocurre a las 16.0 hs con un total de $179014432.64


In [17]:
# Día de la semana (0=Lunes, 6=Domingo)
ventas_enriquecidas['DiaSemana'] = ventas_enriquecidas['SalesDate'].dt.weekday

# Clasificamos según sea fin de semana o entre semana
ventas_enriquecidas['TipoDia'] = ventas_enriquecidas['DiaSemana'].apply(
    lambda x: 'Fin de semana' if x >= 5 else 'Entre semana'
)

# Comparamos total de ventas por grupo
ventas_por_tipo_dia = (
    ventas_enriquecidas.groupby('TipoDia')['TotalPriceCalculated']
    .sum()
    .reset_index()
    .sort_values(by='TotalPriceCalculated', ascending=False)
)

display(ventas_por_tipo_dia)

ganador = ventas_por_tipo_dia.iloc[0]
print(f"📈 Se vende más durante: **{ganador['TipoDia']}**, con un total de ${ganador['TotalPriceCalculated']:.2f}")

,TipoDia,TotalPriceCalculated
0,Entre semana,3.123405e+09
1,Fin de semana,1.192863e+09


📈 Se vende más durante: **Entre semana**, con un total de $3123404917.75


### PI 4 -------------------------------------------

In [18]:
# Cargamos el archivo con empleados
ruta_employees = r"D:\SoyHenry\Proyecto integrador\data\employees.csv"
employees = pd.read_csv(ruta_employees)

# Convertimos las fechas a tipo datetime
employees['BirthDate'] = pd.to_datetime(employees['BirthDate'])
employees['HireDate'] = pd.to_datetime(employees['HireDate'])

# Vista rápida
employees.head()

,EmployeeID,FirstName,MiddleInitial,LastName,BirthDate,Gender,CityID,HireDate
0,1,Nicole,T,Fuller,1981-03-07,F,80,2011-06-20 07:15:36.920
1,2,Christine,W,Palmer,1968-01-25,F,4,2011-04-27 04:07:56.930
2,3,Pablo,Y,Cline,1963-02-09,M,70,2012-03-30 18:55:23.270
3,4,Darnell,O,Nielsen,1989-02-06,M,39,2014-03-06 06:55:02.780
4,5,Desiree,L,Stuart,1963-05-03,F,23,2014-11-16 22:59:54.720


In [19]:
# Hacemos el merge con la tabla de empleados
ventas_empleados = ventas_enriquecidas.merge(
    employees[['EmployeeID', 'BirthDate', 'HireDate']],
    left_on='SalesPersonID',
    right_on='EmployeeID',
    how='left'
)

In [20]:
# Edad al momento de la contratación
ventas_empleados['EdadContratacion'] = (
    (ventas_empleados['HireDate'] - ventas_empleados['BirthDate'])
    .dt.days // 365
)

# Años de experiencia al momento de la venta
ventas_empleados['AniosExperiencia'] = (
    (ventas_empleados['SalesDate'] - ventas_empleados['HireDate'])
    .dt.days // 365
)

# Vista previa
ventas_empleados[['SalesID', 'SalesPersonID', 'EdadContratacion', 'AniosExperiencia']].head()

,SalesID,SalesPersonID,EdadContratacion,AniosExperiencia
0,1,6,26,4.0
1,2,16,65,0.0
2,3,13,48,6.0
3,4,8,57,3.0
4,5,10,48,5.0


### PI 5 -------------------------------------------

In [21]:
# Partimos del dataset ventas_empleados (ya tiene TotalPriceCalculated + experiencia)
dataset_modelado = ventas_empleados.copy()

# Seleccionamos columnas clave
columnas_utiles = [
    'SalesID', 'SalesPersonID', 'CustomerID', 'ProductID', 'Quantity', 'Discount',
    'TotalPriceCalculated', 'HoraVenta', 'DiaSemana', 'TipoDia',
    'EdadContratacion', 'AniosExperiencia'
]

dataset_final = dataset_modelado[columnas_utiles].dropna().copy()

In [22]:
# TipoDia a variables dummies
dataset_final = pd.get_dummies(dataset_final, columns=['TipoDia'], drop_first=True)

# Aseguramos tipo entero en IDs
dataset_final[['SalesPersonID', 'CustomerID', 'ProductID']] = dataset_final[['SalesPersonID', 'CustomerID', 'ProductID']].astype('int')

In [23]:
ruta_final = r"D:\SoyHenry\Proyecto integrador\data\dataset_modelado.csv"
dataset_final.to_csv(ruta_final, index=False)
print(f"✅ Dataset definitivo guardado en:\n{ruta_final}")

✅ Dataset definitivo guardado en:
D:\SoyHenry\Proyecto integrador\data\dataset_modelado.csv


In [24]:
# Vista previa del dataframe final
dataset_final.head()

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPriceCalculated,HoraVenta,DiaSemana,EdadContratacion,AniosExperiencia,TipoDia_Fin de semana
0,1,6,27039,381,7,0.0,309.64,7.0,0.0,26,4.0,False
1,2,16,25011,61,7,0.0,437.82,16.0,4.0,65,0.0,False
2,3,13,94024,23,24,0.0,1896.44,19.0,3.0,48,6.0,False
3,4,8,73966,176,19,0.2,1236.01,14.0,5.0,57,3.0,True
4,5,10,32653,310,9,0.0,719.80,15.0,0.0,48,5.0,False
